In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [5]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [59]:
# Generator Code
#nc - number of color channels in the input images. For color images this is 3
#nz - length of latent vector (i.e. size of generator input)
#ngf - relates to the depth of feature maps carried through the generator (Size of feature maps in generator)
#ndf - sets the depth of feature maps propagated through the discriminator (Size of feature maps in discriminator)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

nc=1 #channel이랄 것이 딱히 없음


class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, (2,4), (1,0), bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 16
            
            nn.ConvTranspose2d( ngf * 2, ngf , 4, 2, 1, bias=False, dilation =(1,2)),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 35
            nn.ConvTranspose2d( ngf , 1, 4, 2, (2,0), bias=False, dilation=(1,2)),
            nn.Tanh()
            # state size. (ngf) x 32 x 75
            
        )
    
    
    def forward(self, input):
        return self.main(input)

In [49]:
model=DCGANGenerator()

In [54]:
#real_cpu = data[0].to(device)
#b_size = real_cpu.size(0)""
b_size=3  #batch size
noise=torch.randn(1, nz, 1, 1)

In [60]:
noise.size()

torch.Size([1, 100, 1, 1])

input tensor를 forward()함수에 넣을 때 여러 개의 input이 batch로 들어갈 수 있다. 

In [61]:
output=model(torch.tensor(noise))

C:\Users\dhk13\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [62]:
output.size()

torch.Size([1, 64, 30, 75])

In [63]:
output

tensor([[[[ 0.0456,  0.0000,  0.0324,  ...,  0.4372,  0.0000,  0.0639],
          [-0.0099,  0.0000, -0.0803,  ...,  0.2237,  0.0000,  0.0123],
          [ 0.0456,  0.0000, -0.2163,  ...,  0.3680,  0.0000,  0.0639],
          ...,
          [-0.0099,  0.0000,  0.2438,  ...,  0.2713,  0.0000,  0.0123],
          [ 0.0456,  0.0000,  0.2828,  ...,  0.3887,  0.0000,  0.0639],
          [-0.0099,  0.0000, -0.0299,  ..., -0.1912,  0.0000,  0.0123]],

         [[-0.0668,  0.0000, -0.1088,  ..., -0.1823,  0.0000, -0.0984],
          [-0.0100,  0.0000, -0.1803,  ...,  0.0246,  0.0000,  0.0805],
          [-0.0668,  0.0000, -0.1820,  ..., -0.0057,  0.0000, -0.0984],
          ...,
          [-0.0100,  0.0000, -0.1750,  ...,  0.1109,  0.0000,  0.0805],
          [-0.0668,  0.0000, -0.0957,  ..., -0.0062,  0.0000, -0.0984],
          [-0.0100,  0.0000, -0.2264,  ...,  0.2025,  0.0000,  0.0805]],

         [[ 0.0445,  0.0000, -0.3060,  ..., -0.1446,  0.0000, -0.0914],
          [-0.0485,  0.0000, -

In [6]:
arr=np.load('./snapshot/gan_cnn_generated_action01.npy')

In [7]:
arr.shape

(1, 30, 75)